In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import timm  # pretrained model lib

C:\Users\ASUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:


# ✅ CONFIG
IMG_DIR = 'image/'
TRAIN_CSV = 'train.csv'
TEST_CSV = 'sample_submission.csv'
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LABELS = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum',
          'Fracture', 'Lung Lesion', 'Lung Opacity', 'No Finding']

# ✅ Dataset
class ChestXrayDataset(Dataset):
    def __init__(self, df, img_dir, transform=None, is_train=True):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        self.is_train = is_train

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(os.path.join(self.img_dir, row['filename'])).convert("RGB")
        if self.transform:
            image = self.transform(image)
        if self.is_train:
            label = torch.tensor(row[LABELS].values.astype(float), dtype=torch.float32)
            return image, label
        else:
            return image

# ✅ Transforms
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# ✅ Load data
df_train = pd.read_csv(TRAIN_CSV)
df_test = pd.read_csv(TEST_CSV)

train_dataset = ChestXrayDataset(df_train, IMG_DIR, transform, is_train=True)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

test_dataset = ChestXrayDataset(df_test, IMG_DIR, transform, is_train=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# ✅ Model
model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=len(LABELS))
model = model.to(DEVICE)

# ✅ Loss & Optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# ✅ Training loop
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {total_loss/len(train_loader):.4f}")

# ✅ Inference
model.eval()
predictions = []
with torch.no_grad():
    for imgs in test_loader:
        imgs = imgs.to(DEVICE)
        outputs = model(imgs)
        preds = torch.sigmoid(outputs).cpu().numpy()
        predictions.append(preds)

preds = np.vstack(predictions)
preds_binary = (preds > 0.5).astype(int)  # Threshold = 0.5

# ✅ Create submission
submission_df = df_test.copy()
submission_df[LABELS] = preds_binary
submission_df.to_csv("submission.csv", index=False)
print("✅ Done! Submission saved to submission.csv")
